In [5]:
import pandas as pd
import sys 
import os 
# Add the parent directory to the Python path 
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from scripts.data_cleaning import get_xdr_data, clean_xdr_data

In [7]:
# Get data from PostgreSQL
data = get_xdr_data()

In [9]:
import numpy as np
from sklearn.cluster import KMeans 
from sklearn.metrics.pairwise import euclidean_distances
# Preprocess the data as done previously
data.fillna(data.mean(), inplace=True)
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.fillna(data.mean(), inplace=True)

# Aggregating per customer
aggregated_data = data.groupby('MSISDN/Number').agg({
    'TCP DL Retrans. Vol (Bytes)': 'mean',
    'TCP UL Retrans. Vol (Bytes)': 'mean',
    'Avg RTT DL (ms)': 'mean',
    'Avg RTT UL (ms)': 'mean',
    'Avg Bearer TP DL (kbps)': 'mean',
    'Avg Bearer TP UL (kbps)': 'mean',
    'Handset Type': 'first'
}).reset_index()

# Compute average retransmission and RTT
aggregated_data['Average TCP Retransmission'] = (aggregated_data['TCP DL Retrans. Vol (Bytes)'] + aggregated_data['TCP UL Retrans. Vol (Bytes)']) / 2
aggregated_data['Average RTT'] = (aggregated_data['Avg RTT DL (ms)'] + aggregated_data['Avg RTT UL (ms)']) / 2
aggregated_data['Average Throughput'] = (aggregated_data['Avg Bearer TP DL (kbps)'] + aggregated_data['Avg Bearer TP UL (kbps)']) / 2

# Drop the intermediate columns
aggregated_data.drop(columns=['TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)', 'Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)'], inplace=True)

# Perform k-means clustering to identify cluster centers
features = aggregated_data[['Average TCP Retransmission', 'Average RTT', 'Average Throughput']]
kmeans = KMeans(n_clusters=3, random_state=42)
aggregated_data['Cluster'] = kmeans.fit_predict(features)

# Getting the cluster centers
cluster_centers = kmeans.cluster_centers_

# Engagement score: Euclidean distance to the less engaged cluster (Cluster 0)
less_engaged_cluster_center = cluster_centers[0]
aggregated_data['Engagement Score'] = euclidean_distances(
    aggregated_data[['Average TCP Retransmission', 'Average RTT', 'Average Throughput']], 
    less_engaged_cluster_center.reshape(1, -1)
)

# Experience score: Euclidean distance to the worst experience cluster (assume it is Cluster 2)
worst_experience_cluster_center = cluster_centers[2]
aggregated_data['Experience Score'] = euclidean_distances(
    aggregated_data[['Average TCP Retransmission', 'Average RTT', 'Average Throughput']], 
    worst_experience_cluster_center.reshape(1, -1)
)

aggregated_data.head()


/tmp/ipykernel_26150/1511952950.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data.fillna(data.mean(), inplace=True)
/tmp/ipykernel_26150/1511952950.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data.fillna(data.mean(), inplace=True)


,MSISDN/Number,Handset Type,Average TCP Retransmission,Average RTT,Average Throughput,Cluster,Engagement Score,Experience Score
0,3.360100e+10,Huawei P20 Lite Huawei Nova 3E,1.078479e+07,23.000000,38.0,0,2.561888e+06,4.865116e+08
1,3.360100e+10,Apple iPhone 7 (A1778),1.078479e+07,15.500000,49.5,0,2.561888e+06,4.865116e+08
2,3.360100e+10,undefined,1.078479e+07,63.729294,48.5,0,2.561888e+06,4.865116e+08
3,3.360101e+10,Apple iPhone 5S (A1457),3.803623e+05,42.000000,124.0,0,7.842547e+06,4.969160e+08
4,3.360101e+10,Apple iPhone Se (A1723),7.735101e+06,29.750000,14211.0,0,4.878663e+05,4.895613e+08


In [10]:
# Compute satisfaction score
aggregated_data['Satisfaction Score'] = (aggregated_data['Engagement Score'] + aggregated_data['Experience Score']) / 2

# Report top 10 satisfied customers
top_10_satisfied = aggregated_data.nlargest(10, 'Satisfaction Score')

print("Top 10 Satisfied Customers:")
print(top_10_satisfied[['MSISDN/Number', 'Satisfaction Score']])


Top 10 Satisfied Customers:
       MSISDN/Number  Satisfaction Score
85785   3.369858e+10        1.897979e+09
32273   3.365871e+10        1.891301e+09
58026   3.366491e+10        1.881564e+09
31942   3.365863e+10        1.874570e+09
62049   3.366613e+10        1.852869e+09
70079   3.366877e+10        1.830538e+09
43629   3.366131e+10        1.816209e+09
48341   3.366240e+10        1.806143e+09
64113   3.366682e+10        1.731301e+09
98211   3.376264e+10        1.640676e+09


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Task 4.3: Build regression model
# Features and target
X = aggregated_data[['Average TCP Retransmission', 'Average RTT', 'Average Throughput', 'Engagement Score', 'Experience Score']]
y = aggregated_data['Satisfaction Score']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and train the model
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

# Predict on test set
y_pred = reg_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")


Mean Squared Error: 1.7116371192549048e-15
R2 Score: 1.0


In [12]:
# Run k-means (k=2) on engagement and experience scores
kmeans_2 = KMeans(n_clusters=2, random_state=42)
aggregated_data['Engagement-Experience Cluster'] = kmeans_2.fit_predict(aggregated_data[['Engagement Score', 'Experience Score']])

aggregated_data.head()


,MSISDN/Number,Handset Type,Average TCP Retransmission,Average RTT,Average Throughput,Cluster,Engagement Score,Experience Score,Satisfaction Score,Engagement-Experience Cluster
0,3.360100e+10,Huawei P20 Lite Huawei Nova 3E,1.078479e+07,23.000000,38.0,0,2.561888e+06,4.865116e+08,2.445367e+08,0
1,3.360100e+10,Apple iPhone 7 (A1778),1.078479e+07,15.500000,49.5,0,2.561888e+06,4.865116e+08,2.445367e+08,0
2,3.360100e+10,undefined,1.078479e+07,63.729294,48.5,0,2.561888e+06,4.865116e+08,2.445367e+08,0
3,3.360101e+10,Apple iPhone 5S (A1457),3.803623e+05,42.000000,124.0,0,7.842547e+06,4.969160e+08,2.523793e+08,0
4,3.360101e+10,Apple iPhone Se (A1723),7.735101e+06,29.750000,14211.0,0,4.878663e+05,4.895613e+08,2.450246e+08,0


In [13]:
# Aggregate average satisfaction and experience score per cluster
cluster_agg = aggregated_data.groupby('Engagement-Experience Cluster').agg({
    'Satisfaction Score': 'mean',
    'Experience Score': 'mean'
}).reset_index()

print("Average Satisfaction and Experience Score Per Cluster:")
print(cluster_agg)


Average Satisfaction and Experience Score Per Cluster:
   Engagement-Experience Cluster  Satisfaction Score  Experience Score
0                              0        2.469327e+08      4.884576e+08
1                              1        9.757290e+08      7.311923e+08


In [19]:
import psycopg2

# Define a function to export data to PostgreSQL
def export_to_postgresql(df):
    # PostgreSQL connection parameters
    conn_params = {
        'dbname': 'postgres',
        'user': 'postgres',
        'password': 'postgres',
        'host': 'localhost',
        'port': '5432'
    }
    
    # Establish a connection to PostgreSQL
    conn = psycopg2.connect(**conn_params)
    
    # Create a cursor object
    cursor = conn.cursor()
    
    # Create table statement
    create_table_query = '''
    CREATE TABLE IF NOT EXISTS user_engagement_analysis (
        msisdn_number TEXT,
        handset_type TEXT,
        average_tcp_retransmission FLOAT,
        average_rtt FLOAT,
        average_throughput FLOAT,
        cluster INT,
        engagement_score FLOAT,
        experience_score FLOAT,
        satisfaction_score FLOAT,
        engagement_experience_cluster INT
    );
    '''
    cursor.execute(create_table_query)
    conn.commit()
    
    # Insert data into the table
    for _, row in df.iterrows():
        insert_query = '''
        INSERT INTO user_engagement_analysis (msisdn_number, handset_type, average_tcp_retransmission, average_rtt, average_throughput, cluster, engagement_score, experience_score, satisfaction_score, engagement_experience_cluster)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        '''
        cursor.execute(insert_query, tuple(row))
    
    conn.commit()
    
    # Select query for screenshot
    select_query = "SELECT * FROM user_engagement_analysis LIMIT 10;"
    cursor.execute(select_query)
    result = cursor.fetchall()
    
    # Close the connection
    cursor.close()
    conn.close()
    
    return result

# Export the final table
result = export_to_postgresql(aggregated_data)

# Print the result of the select query
print(result)


AttributeError: 'Engine' object has no attribute 'cursor'